In [1]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import plot_tree
from sklearn.metrics import roc_auc_score,accuracy_score,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.naive_bayes import GaussianNB



import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("HR_comma_sep.csv")
X= df.drop('left', axis = 1)
y= df['left']

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

CatBoost Without One Hot Encoding 

In [4]:
cgbm= CatBoostClassifier(random_state=24, cat_features=['Department', 'salary'], n_estimators= 50 )
cgbm.fit(X_train, y_train)

Learning rate set to 0.438486
0:	learn: 0.4127012	total: 194ms	remaining: 9.51s
1:	learn: 0.2883445	total: 229ms	remaining: 5.5s
2:	learn: 0.2171350	total: 268ms	remaining: 4.2s
3:	learn: 0.1787252	total: 303ms	remaining: 3.48s
4:	learn: 0.1554308	total: 337ms	remaining: 3.03s
5:	learn: 0.1402544	total: 381ms	remaining: 2.79s
6:	learn: 0.1274649	total: 419ms	remaining: 2.57s
7:	learn: 0.1223568	total: 459ms	remaining: 2.41s
8:	learn: 0.1151863	total: 494ms	remaining: 2.25s
9:	learn: 0.1090791	total: 524ms	remaining: 2.1s
10:	learn: 0.1055714	total: 564ms	remaining: 2s
11:	learn: 0.1027904	total: 593ms	remaining: 1.88s
12:	learn: 0.1001674	total: 621ms	remaining: 1.77s
13:	learn: 0.0980211	total: 651ms	remaining: 1.67s
14:	learn: 0.0960815	total: 681ms	remaining: 1.59s
15:	learn: 0.0946345	total: 712ms	remaining: 1.51s
16:	learn: 0.0927250	total: 740ms	remaining: 1.44s
17:	learn: 0.0906349	total: 771ms	remaining: 1.37s
18:	learn: 0.0885028	total: 799ms	remaining: 1.3s
19:	learn: 0.08712

In [5]:
y_pred = cgbm.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9773282951767059


In [6]:
kFold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)
params = {'max_depth' : [2,3,4], 'n_estimators':[10,50],'learning_rate':np.linspace(0.001,1,5) }
gcv = GridSearchCV(cgbm,param_grid=params,cv=kFold,scoring='roc_auc',verbose=3)
gcv.fit(X,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
0:	learn: 0.6925472	total: 10.3ms	remaining: 92.3ms
1:	learn: 0.6919348	total: 25.4ms	remaining: 102ms
2:	learn: 0.6913254	total: 37.7ms	remaining: 88ms
3:	learn: 0.6907172	total: 46.3ms	remaining: 69.5ms
4:	learn: 0.6901085	total: 55.3ms	remaining: 55.3ms
5:	learn: 0.6895010	total: 65.9ms	remaining: 44ms
6:	learn: 0.6888947	total: 76.8ms	remaining: 32.9ms
7:	learn: 0.6882913	total: 87.7ms	remaining: 21.9ms
8:	learn: 0.6876875	total: 98.7ms	remaining: 11ms
9:	learn: 0.6870981	total: 110ms	remaining: 0us
[CV 1/5] END learning_rate=0.001, max_depth=2, n_estimators=10;, score=0.911 total time=   0.1s
0:	learn: 0.6925453	total: 15.3ms	remaining: 138ms
1:	learn: 0.6919295	total: 29.8ms	remaining: 119ms
2:	learn: 0.6913171	total: 45.5ms	remaining: 106ms
3:	learn: 0.6907059	total: 57.6ms	remaining: 86.5ms
4:	learn: 0.6900937	total: 70.4ms	remaining: 70.4ms
5:	learn: 0.6894828	total: 85.5ms	remaining: 57ms
6:	learn: 0.6888731	total:

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x0000020760F5F770>,
             param_grid={'learning_rate': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'max_depth': [2, 3, 4], 'n_estimators': [10, 50]},
             scoring='roc_auc', verbose=3)

In [7]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 1.0, 'max_depth': 4, 'n_estimators': 50}
0.9893002633546979
